# Lecture 8: More about Monte Carlo methods

## Physics 7810, Spring 2020

## 8.1 - Trial moves
 
For lattice models such as the Ising model, simple discrete Monte Carlo moves such as spin flips can be used. It's easy to verify that such moves satisfy detailed balance, as the proposal probability $\alpha$ is manifestly symmetric,

$$
\alpha(o \to n) = \alpha(n \to o).
$$

In other words, the probability of proposing to flip a spin in a given MC move is equal to the probability of proposing to 'unflip' it in the next MC move.

It can be more difficult to design trial moves that preserve detailed balance in more complex situations, e.g., for off-lattice models such as the Lennard-Jones or hard sphere systems.

## 8.2 - Translational moves
 
For off-lattice models, it's conventional to generate trial particle displacements within a cubic volume of edge length $\Delta$ centered on the current particle position,

$$
\begin{eqnarray}
x_i^\prime & = & x_i + \Delta \left[ \mathrm{unif}(0,1) - 1/2 \right] \\
y_i^\prime & = & y_i + \Delta \left[ \mathrm{unif}(0,1) - 1/2 \right] \\
z_i^\prime & = & z_i + \Delta \left[ \mathrm{unif}(0,1) - 1/2 \right],
\end{eqnarray}
$$

where $\mathrm{unif}(0,1)$ denotes a continuous random variable uniformly distributed on the interval $(0,1)$.

It's clear that the reverse trial move is equally probable, so this choice satisfies the detailed balance requirement.  A more symmetric choice would be to generate displacements within a *spherical* volume centered on the current particle position, but this procedure isn't commonly used. Why not?

This trial move involves a free parameter $\Delta$, which can be adjusted to optimize the MC procedure, as we will discuss shortly.

(max displacement is $0.5 \Delta$?)

In a conventional translational trial move, state $n$ is obtained from state $o$ by generating a particle displacement  anywhere within the shaded region:

<img src="images/A&T_Fig_4.2.png" alt="Drawing" style="width: 500px;">

Figure from *Computer Simulation of Liquids*, by Mike Allen and Dominic Tildesley.

How do we gauge the efficiency of a given MC algorithm (e.g., as a function of $\Delta$)?

We can generally define the efficiency of a method as:

$$
\mathrm{efficiency} = \frac{\mathrm{Number\ of\ statistically\ independent\ configurations\ generated}}{\mathrm{CPU\ time}}.
$$

Note that this definition applies equally well to MD and MC simulations. Making use of this definition requires understanding what we mean by *statistically independent*, and developing ways of measuring the number of statistically independent samples generated in a given simulation.

Also, note that a simulation procedure may be optimal for sampling one quantity, but non-optimal for sampling another. For example, an algorithm for simulating non-spherical molecules may be optimal for sampling *translational* degrees for freedom, but not for sampling *orientational* degrees of freedom.

*Collective* MC moves that involve updating many degrees of freedom simultaneously are usually (but not always) highly inefficient, because most such trial moves will be rejected.

As described in Frenkel and Smit, a crude but generally effective way of optimizing translational trial moves with respect to $\Delta$ involves estimating the rate at which statistically independent samples are generated by measuring a quantity related to the mean-squared particle displacement (a measure of the rate at which new configurations are explored).

The mean-squared displacement of a particle in a time interval $\tau$ is defined as

$$
\left\langle \Delta {\bf r}_i^2(\tau)  \right\rangle
= \left\langle \left[ {\bf r}_i(t + \tau) - {\bf r}_i(t) \right]^2 \right\rangle,
$$

where the angle brackets denote an average over all particles $i$ and all time origins $t$. In a fluid phase in which particles can diffuse, the mean-squared displacement is related to the diffusion constant $D$ by

$$
\left\langle \Delta {\bf r}_i^2(\tau)  \right\rangle = 2 d D \tau
$$

for large $\tau$, where $d$ is the spatial dimensionality. In the absence of diffusion, for example in a crystalline or glassy solid, this isn't a useful measure of sampling efficiency. For this purpose we need to define slightly different quantity, and we need to clarify what we mean by 'time' in an MC simulation.

To estimate the sampling efficiency of an MC algorithm, we compute an $\mathrm{msd}$-like quantity, the single-move mean-squared particle displacements averaged over MC moves,  

$$
\left\langle \Delta {\bf r}_i^2 \right\rangle,
$$

where the angle brackets now denote an average over all particles and over all trial MC moves. Note that only *accepted* moves make a nonzero contribution to this average, but the average is over *all* trial moves (accepted or not). Thus, $\left\langle \Delta {\bf r}_i^2 \right\rangle$ will be small for small $\Delta$ (where most trial moves are accepted, but only generate small displacements) and for large $\Delta$ (where most trial moves are rejected).

We can now define a diffusion-constant-like measure of efficiency based on $\left\langle \Delta {\bf r}_i^2 \right\rangle$, namely

$$
\mathrm{efficiency} = \frac{\left\langle \Delta {\bf r}_i^2 \right\rangle}{\mathrm{CPU\ time}}.
$$

This measure of efficiency can be used for 'on-the-fly' optimization of MC simulations, although such optimizations shouldn't be carried out during *production* runs, as they typically (weakly) violate detailed balance.

Qualitative dependence of the single-move mean-squared displacement on the average size $\Delta$ of trial moves (left), and dependence of the computational cost per move on $\Delta$ for continuous and hard-core potentials (right). For hard-core systems, an 'early exit' strategy leads to a decrease in computational cost with increasing $\Delta$.
 
<img src="images/Frenkel_Fig_3.4.png" alt="Drawing" style="width: 1000px;">

Figure from *Understanding Molecular Simulation: from Algorithms to Applications*, by Daan Frenkel and Berend Smit.

## 8.3 - Pair distribution function
 
The structure of condensed phases can be characterized by a set of average $n$-particle distribution functions that measure the probability of finding $n$ particles with specific positions ${\bf r}^n$, and hence probe $n$-particle *correlations*. The simplest such distribution function is the *pair distribution function* $g_2({\bf r}_1, {\bf r}_1) \equiv g({\bf r}_1, {\bf r}_1)$, which is the probability of finding a distance $r = |{\bf r}_2 - {\bf r}_1|$ apart relative to the probability for a random distribution of particles at the same density.

In the canonical ensemble, the pair distribution function can be written as

$$
g({\bf r}_1, {\bf r}_1) = \frac{N(N-1)}{\rho^2 Z^\mathrm{int}} \int d{\bf r}_3 d{\bf r}_4 ... d{\bf r}_N
\exp \left[ - \beta U({\bf r}^N) \right].
$$

See, e.g., *Theory of Simple Liquids*, by J. P. Hansen and I. R. McDonald.

Equivalently, we can define the pair distribution function starting with the pair density for particles separated by a distance ${\bf r}$,

$$
\rho^{(2)}({\bf r}^\prime + {\bf r}, {\bf r}^\prime) = \left\langle \sum_i \sum_{j \neq i}
\delta({\bf r}^\prime + {\bf r} - {\bf r}_i) \delta({\bf r}^\prime - {\bf r}_j) \right\rangle.
$$

In a homogeneous system, this is independent of ${\bf r}^\prime$. In this case, $g({\bf r})$ is given by

$$
g({\bf r}) = \frac{\rho^{(2)}({\bf r}^\prime + {\bf r}, {\bf r}^\prime)}{\rho^2}
= \frac{V^2}{N^2} \frac{1}{V} \int d{\bf r}^\prime \rho^{(2)}({\bf r}^\prime + {\bf r}, {\bf r}^\prime).
$$

Here $\rho = N/V$ is the particle density, and we've averaged over ${\bf r}^\prime$ in the last step.
(Note that the single particle density is $\rho^{(1)}(\vec{r})=\langle \sum_{i=1}^{N} \delta(\vec{r}-\vec{r_{i}})\rangle$.)

Integrating over ${\bf r}^\prime$ eliminates one of the delta functions, giving

$$
g({\bf r}) = \frac{V}{N^2} \left\langle \sum_i \sum_{j \neq i}
\delta({\bf r} - {\bf r}_{ij}) \right\rangle,
$$

where ${\bf r}_{ij} = {\bf r}_i - {\bf r}_j$.

In an isotropic fluid, $g({\bf r})$ only depends on $r = |{\bf r}|$, i.e.,

$$
g(r) = \frac{V}{N^2} \left\langle \sum_i \sum_{j \neq i}
\delta({\bf r} - {\bf r}_{ij}) \right\rangle.
$$

Pair distribution function for the Lennard-Jones fluid close to its triple point ($T^\ast = 0.8$, $\rho^\ast = 0.8$). Note that $\lim_{r \to \infty} g(r) = 1$.
 
<img src="images/A&T_Fig_2.2.png" alt="Drawing" style="width: 800px;">

Figure from *Computer Simulation of Liquids*, by Mike Allen and Dominic Tildesley.

Pair distribution function of argon near its triple point, obtained from neutron scattering experiments. The ripples at small $r$ are an artifact of the data analysis.
 
<img src="images/H&M_Fig_2.3.png" alt="Drawing" style="width: 700px;">

Figure from *Theory of Simple Liquids*, by J. P. Hansen and I. R. McDonald.

Because the pair distribution function is a measure of pair correlations, the ensemble average of any pair function $a({\bf r}_i, {\bf r}_j)$ can be written in terms of $g({\bf r}_i, {\bf r}_j)$,

$$
\left\langle a({\bf r}_i, {\bf r}_j) \right\rangle = \frac{1}{V^2} \int d{\bf r}_i d{\bf r}_j a({\bf r}_i, {\bf r}_j) g({\bf r}_i, {\bf r}_j).
$$

For an isotropic fluid, we have

$$
\left\langle \sum_i \sum_{j>i} a(r_{ij}) \right\rangle = \frac{1}{2} N \rho \int_0^\infty dr 4 \pi r^2 a(r) g(r).
$$

Thus, for example, the average potential energy per particle can be written

$$
\left\langle \frac{U}{N} \right\rangle
= \left\langle \frac{1}{N} \sum_i \sum_{j>i} u(r_{ij}) \right\rangle = 2 \pi \rho \int_0^\infty dr r^2 u(r) g(r).
$$

Similarly, the average pressure can be written

$$
\left\langle PV \right\rangle
= N k_B T - \frac{2}{3} \pi N \rho \int_0^\infty dr r^2 w(r) g(r).
$$

Here, $w(r) = r\ du(r)/dr$ is the *pair virial*, in terms of which the total virial defined earlier can be expressed as

$$
W = \frac{1}{3} \sum_i \sum_{j>i} {\bf r}_{ij} \cdot {\bf r}_{ij}
= - \frac{1}{3} \sum_i \sum_{j>i} {\bf r}_{ij} \cdot \nabla_{{\bf r}_{ij}} u(r_{ij})
= - \frac{1}{3} \sum_i \sum_{j>i} {\bf r}_{ij} \cdot \left[ \frac{du(r_{ij})}{dr_{ij}} \nabla_{{\bf r}_{ij}} r_{ij} \right]
$$
$$
= - \frac{1}{3} \sum_i \sum_{j>i} {\bf r}_{ij} \cdot \left[ \frac{du(r_{ij})}{dr_{ij}} \frac{{\bf r}_{ij}}{r_{ij}} \right]
= - \frac{1}{3} \sum_i \sum_{j>i} w(r_{ij}).
$$

It's generally more accurate to compute these quantities directly, but these expressions are useful for obtaining long-range corrections in simulations with an interaction cutoff.

## 8.4 - Long-range corrections
 
Truncation of pair interactions using cutoffs introduces a dependence of the simulation results on the interaction cutoff $r_c$. It's desirable to remove this artifical dependence on $r_c$ if at all possible. To a first approximation, we can do this by computing long-range corrections to average thermodynamic quantities such as the potential energy and pressure, assuming that the density of interacting pairs is uniform beyond the interaction cutoff $r_c$, i.e., $g(r) \cong 1$ for $r > r_c$.

In this case, the long-range ('tail') correction to the potential energy per particle is

$$
u^\mathrm{tail} = \frac{U^\mathrm{tail}}{N} = 2 \pi \rho \int_{r_c}^\infty dr r^2 u(r),
$$

and the long-range correction to the pressure is

$$
P^\mathrm{tail} = - \frac{2}{3} \pi \rho^2 \int_{r_c}^\infty dr r^2 w(r).
$$

For the Lennard-Jones potential,

$$
u_\mathrm{LJ}(r) = 4 \epsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^6 \right],
$$

we obtain

$$
u^\mathrm{tail} = \frac{8 \pi}{3} \epsilon \rho \sigma^3 \left[ \frac{1}{3} \left( \frac{\sigma}{r_c} \right)^9 - \left( \frac{\sigma}{r_c} \right)^3 \right]
$$

and

$$
P^\mathrm{tail} = \frac{16 \pi}{3} \epsilon \rho^2 \sigma^3 \left[ \frac{2}{3} \left( \frac{\sigma}{r_c} \right)^9 - \left( \frac{\sigma}{r_c} \right)^3 \right]$$

For molecular dynamics simulations with a truncated and *shifted* potential, an additional term should be added to the average potential energy per particle to correct for the shift in the potential,

$$
u^\mathrm{shift} = \frac{1}{2} \langle n(r < r_c)\ \rangle u(r_c),
$$

where $\langle n(r < r_c) \rangle$ is the average number of particles within a distance $r_c$ from a given particle, $u(r_c)$ is the value of the (untruncated, unshifted) pair potential at the cutoff, and the factor of $1/2$ is included to avoid overcounting pair interactions.

Because the forces aren't changed by a shift in the potential, there's not corresponding correction to the pressure.

Note that many MD simulation packages use more complicated switching functions instead of sharp cutoffs, so the long-range corrections are correspondingly more complicated.


## 8.5 - Orientational moves
 
For non-spherical particles (e.g., molecules), or for particles with anisotropic interaction potentials (e.g. *dipolar* spheres), we must include trial moves that change the particle orientation in our Monte Carlo procedure.